<a href="https://colab.research.google.com/github/monker490/musical_mood/blob/main/GetTracks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install ffmpeg -y
!pip install pydub

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [ ]:
# Cell 1: Imports and Configuration
import requests
import base64
import hashlib
import secrets
import urllib.parse
import webbrowser
from IPython.display import display, HTML
import time

# Configuration
client_id = '80de2644106e4adbae439a42b7b0b96e'  # Replace with your Spotify Client ID
redirect_uri = 'http://localhost:8888/callback/'  # Must match your Spotify app settings
authorization_endpoint = "https://accounts.spotify.com/authorize"
token_endpoint = "https://accounts.spotify.com/api/token"
scope = 'user-read-private user-read-email'

# Cell 2: PKCE Helper Functions
def generate_code_verifier():
    """Generate a random code verifier string"""
    return secrets.token_urlsafe(64)

def generate_code_challenge(code_verifier):
    """Generate code challenge from verifier using SHA256"""
    sha256_hash = hashlib.sha256(code_verifier.encode()).digest()
    base64_encoded = base64.urlsafe_b64encode(sha256_hash).decode('utf-8')
    return base64_encoded.rstrip('=')

# Cell 3: Generate Authorization URL
code_verifier = generate_code_verifier()
code_challenge = generate_code_challenge(code_verifier)

auth_params = {
    'client_id': client_id,
    'response_type': 'code',
    'redirect_uri': redirect_uri,
    'scope': scope,
    'code_challenge_method': 'S256',
    'code_challenge': code_challenge
}

auth_url = f"{authorization_endpoint}?{urllib.parse.urlencode(auth_params)}"
display(HTML(f'<a href="{auth_url}" target="_blank">Click here to authenticate with Spotify</a>'))
print("After clicking the link, you'll be redirected to a URL with a code parameter.")
print("Copy the full URL and paste it in the next cell.")




After clicking the link, you'll be redirected to a URL with a code parameter.
Copy the full URL and paste it in the next cell.


In [ ]:
# Cell 4: Input Callback URL
callback_url = input("Paste the callback URL here: ")
code = urllib.parse.parse_qs(urllib.parse.urlparse(callback_url).query)['code'][0]
print(f"Extracted code: {code}")

# Cell 5: Exchange Code for Token
token_params = {
    'client_id': client_id,
    'grant_type': 'authorization_code',
    'code': code,
    'redirect_uri': redirect_uri,
    'code_verifier': code_verifier
}

response = requests.post(token_endpoint, data=token_params)
token_data = response.json()

if 'access_token' in token_data:
    access_token = token_data['access_token']
    refresh_token = token_data.get('refresh_token')
    expires_in = token_data['expires_in']
    print("Successfully authenticated!")
    print(f"Access Token: {access_token}")
    print(f"Refresh Token: {refresh_token}")
    print(f"Expires in: {expires_in} seconds")
else:
    print("Error getting token:")
    print(token_data)

# Cell 6: Get User Data
headers = {
    'Authorization': f'Bearer {access_token}'
}
user_response = requests.get('https://api.spotify.com/v1/me', headers=headers)
user_data = user_response.json()

if user_response.status_code == 200:
    display(HTML(f"<h2>Welcome, {user_data.get('display_name', 'User')}!</h2>"))
    print(f"Email: {user_data.get('email', 'Not provided')}")
    print(f"Spotify ID: {user_data['id']}")
else:
    print("Error fetching user data:")
    print(user_data)

Paste the callback URL here: http://localhost:8888/callback/?code=AQA6ppknic4Pj_dBtbYbhB-v6UZmL5dIL4IiGz70H3xrI-4KXickeq1QjX3JRYKH8AzfunLLl5uX7qg597_mY7ZuuIIQStlyQPoAW3EWI0K5g_lzBgQiQkP72YHOUYeMa0iznoZwXl57GEIewvoUR8EAhUjGyN81lZ8jUEgbPjwfsbQ3CtRJdai-zXfx3EGBvm4VppyIcUrTtVG-r7_NJesE-s6Nfrk0X3eem0UZHrjcQqj-sKQgdQmCudYZodrFD6-dVS7HTtzspjvohBiNMeqSJBwFxFqAhb4
Extracted code: AQA6ppknic4Pj_dBtbYbhB-v6UZmL5dIL4IiGz70H3xrI-4KXickeq1QjX3JRYKH8AzfunLLl5uX7qg597_mY7ZuuIIQStlyQPoAW3EWI0K5g_lzBgQiQkP72YHOUYeMa0iznoZwXl57GEIewvoUR8EAhUjGyN81lZ8jUEgbPjwfsbQ3CtRJdai-zXfx3EGBvm4VppyIcUrTtVG-r7_NJesE-s6Nfrk0X3eem0UZHrjcQqj-sKQgdQmCudYZodrFD6-dVS7HTtzspjvohBiNMeqSJBwFxFqAhb4
Successfully authenticated!
Access Token: BQA4gm-ZZJld8LSmKi6C2I5ED_uX9cXO0ujk2XSktIVgui3nf_n9ZHy9g-n67gpbuv5d6icaBcesLAgsze90dey0KSL9_t_G3Wj1b0uwAdQicxtT_0A8MYi0T4PYUDnBjpIX-fcHa_6M35q96HMfBMh9j883JJCw_XbmfVmh1JQC5Y3S5ih7oMW5p_6Iuk-H_BHhnxaR9LMKRLdBbHE3vfeH6DzPe3talA
Refresh Token: AQAeKALjNN0ZPLwrYyUDySnzS70yhe3Bn5nVwzKInaab_dqlp7

Email: arslan.sama@gmail.com
Spotify ID: aryslansid


In [ ]:
# Cell 7: Refresh Token (optional)
def refresh_access_token(refresh_token):
    refresh_params = {
        'client_id': client_id,
        'grant_type': 'refresh_token',
        'refresh_token': refresh_token
    }

    response = requests.post(token_endpoint, data=refresh_params)
    new_token_data = response.json()

    if 'access_token' in new_token_data:
        print("Token refreshed successfully!")
        print(f"New Access Token: {new_token_data['access_token']}")
        return new_token_data['access_token']
    else:
        print("Error refreshing token:")
        print(new_token_data)
        return None

In [ ]:
import requests

#Specify the base URL and the endpoint to get access the API
base_url = 'https://api.spotify.com/v1/'

headers = {
    'Authorization': 'Bearer {}'.format(access_token)
}


my_playlists_endpoint = 'me/playlists?limit=50'
my_playlists_url = ''.join([base_url, my_playlists_endpoint])

response = requests.get(my_playlists_url,headers=headers)

print(response.json())

playlists = response.json().get('items')

{'href': 'https://api.spotify.com/v1/users/aryslansid/playlists?offset=0&limit=50', 'limit': 50, 'next': None, 'offset': 0, 'previous': None, 'total': 6, 'items': [{'collaborative': False, 'description': '', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/6iLHq7FC0XTlL9KceERQUc'}, 'href': 'https://api.spotify.com/v1/playlists/6iLHq7FC0XTlL9KceERQUc', 'id': '6iLHq7FC0XTlL9KceERQUc', 'images': [{'height': 640, 'url': 'https://mosaic.scdn.co/640/ab67616d00001e02267158422744961d90e942b3ab67616d00001e022e02117d76426a08ac7c174fab67616d00001e02b1ba08578c692ac29e583abcab67616d00001e02dd7f90cd5050545f66d837fe', 'width': 640}, {'height': 300, 'url': 'https://mosaic.scdn.co/300/ab67616d00001e02267158422744961d90e942b3ab67616d00001e022e02117d76426a08ac7c174fab67616d00001e02b1ba08578c692ac29e583abcab67616d00001e02dd7f90cd5050545f66d837fe', 'width': 300}, {'height': 60, 'url': 'https://mosaic.scdn.co/60/ab67616d00001e02267158422744961d90e942b3ab67616d00001e022e02117d76426a08ac7c174fa

In [ ]:
import json

def find_by_property(array, key, value):
    return next(filter(lambda item: item.get(key) == value, array), None)


playlist = find_by_property(playlists, "description", "My liked Songs")

artist_ids = set()

preview_urls = {}
pr = requests.get(base_url + 'playlists/{}/tracks?limit=50'.format(playlist.get('id')), headers=headers)

print(pr.json())
pr_data = pr.json()

#Get all artists from a playlist
if pr_data:
    playlist_data = pr_data.get('items')
    for tr in playlist_data:
        track = tr.get('track')

        # json_formatted_str = json.dumps(track, indent=2)
        # print(json_formatted_str)

        if track:
          if track.get('preview_url'):
            # url = track,get('preview_url')
            preview_urls[track.get('name')] = track.get('preview_url')
          artists = track.get('artists')
          for artist in artists:
              artist_id = artist.get('id')
              artist_name = artist.get('name')
              artist_ids.add((artist_id,artist_name))

print(artist_ids)


{'href': 'https://api.spotify.com/v1/playlists/2eNYwFHdhPosmegJdM7m3J/tracks?offset=0&limit=50', 'items': [{'added_at': '2020-06-17T16:37:44Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/aryslansid'}, 'href': 'https://api.spotify.com/v1/users/aryslansid', 'id': 'aryslansid', 'type': 'user', 'uri': 'spotify:user:aryslansid'}, 'is_local': False, 'primary_color': None, 'track': {'preview_url': None, 'available_markets': [], 'explicit': False, 'type': 'track', 'episode': False, 'track': True, 'album': {'available_markets': [], 'type': 'album', 'album_type': 'album', 'href': 'https://api.spotify.com/v1/albums/4ia0Qf7QdwJ9pA2wOcLI3v', 'id': '4ia0Qf7QdwJ9pA2wOcLI3v', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b2738d6320a29955381c7458542c', 'width': 640}, {'height': 300, 'url': 'https://i.scdn.co/image/ab67616d00001e028d6320a29955381c7458542c', 'width': 300}, {'height': 64, 'url': 'https://i.scdn.co/image/ab67616d000048518d6320a29955381

In [ ]:

sample_map = {}

for id, name in artist_ids:
    artist_request = requests.get(base_url + 'artists/{}/top-tracks?'.format(id), headers=headers)
    print(artist_request.json())
    tracks = artist_request.json().get('tracks')



    for track in tracks:
        track_id = track.get('id')

        track_feature_req = requests.get(base_url + 'audio-features/{}'.format(track_id), headers = headers)

        print(track_feature_req.json())

        break

    break




{'tracks': [{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3pTE9iaJTkWns3mxpNQlJV'}, 'href': 'https://api.spotify.com/v1/artists/3pTE9iaJTkWns3mxpNQlJV', 'id': '3pTE9iaJTkWns3mxpNQlJV', 'name': 'Bombay Bicycle Club', 'type': 'artist', 'uri': 'spotify:artist:3pTE9iaJTkWns3mxpNQlJV'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS', 'SI', 'KR', 'BD', 'PK', 'LK', 'GH', 'KE', 'NG', 'TZ', 'UG', 'AG', 'AM', 'BS', 'BB', 'BZ', 'BT', 'BW', '

In [ ]:
from urllib.request import urlretrieve
import random
import json
import os

directory_path = "/content/songs"
os.makedirs(directory_path, exist_ok=True)

random_songs = []

for _ in range(5):
  idx = random.randrange(len(preview_urls))
  random_songs += [list(preview_urls.items())[idx]]

print(random_songs)


http_resp = []
for song in random_songs:
  save_filename = directory_path + "/" + song[0] + ".mp3"
  http_resp = requests.get(song[1], stream=True)
  with open(save_filename, 'wb') as f:
    for chunk in http_resp.iter_content(chunk_size=1024):
      if chunk:
        f.write(chunk)


[('Filma Solo', 'https://p.scdn.co/mp3-preview/496a5bfcd9d8bf48e4c691c02dad7b450e2139e1?cid=80de2644106e4adbae439a42b7b0b96e'), ('Cherish (My Love)', 'https://p.scdn.co/mp3-preview/e2aee1384762736e0d8fe356e2b7bd06c0a156c1?cid=80de2644106e4adbae439a42b7b0b96e'), ('Amaranth', 'https://p.scdn.co/mp3-preview/4f044a54ffb1bf5098eb19deb20fef19578a378a?cid=80de2644106e4adbae439a42b7b0b96e'), ('Georgia Girl', 'https://p.scdn.co/mp3-preview/1d7d4984656f1377643a6b7ebfa61ac76fe70ff2?cid=80de2644106e4adbae439a42b7b0b96e'), ('John Wayne (feat. Metro Boomin)', 'https://p.scdn.co/mp3-preview/bd79f875eece41dc98e3b7b8bf34a5eb2bd0f0a9?cid=80de2644106e4adbae439a42b7b0b96e')]


In [ ]:
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
import IPython.display as ipd

def remove_silence(input_file, output_file="output.wav", silence_thresh=-40, min_silence_len=500):
    """
    Removes silent parts from an audio file and saves the cleaned audio.

    Parameters:
    - input_file (str): Path to the input audio file.
    - output_file (str): Path to save the cleaned audio file.
    - silence_thresh (int): Volume threshold (in dB) below which sound is considered silence.
    - min_silence_len (int): Minimum length (in ms) of silence to be removed.
    """
    # Load audio file
    audio = AudioSegment.from_file(input_file)

    # Detect non-silent chunks
    nonsilent_chunks = detect_nonsilent(audio, min_silence_len=min_silence_len, silence_thresh=silence_thresh)

    # Extract non-silent parts
    processed_audio = AudioSegment.empty()
    for start, end in nonsilent_chunks:
        processed_audio += audio[start:end]

    # Save cleaned audio
    processed_audio.export(output_file, format="wav")
    print(f"✅ Processed audio saved as {output_file}")

    # Return processed audio for playback
    return output_file

# Example usage: Replace "input.wav" with your file
output_path = remove_silence("input.wav")

# Play the cleaned audio
ipd.Audio(output_path)


In [ ]:
import librosa
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# def prepare_song(song_path, idx, ax):
#   list_matrices = []
#   print(song_path)
#   y, sr = librosa.load(song_path, sr=48000, mono = False)
#   # song_pieces = cut_song(y)
#   # for song_piece in song_pieces:
#   melspect = librosa.feature.melspectrogram(y=y, sr=sr)

#   librosa.display.specshow(librosa.power_to_db(melspect), x_axis='time', y_axis='log', ax=[idx,0])
#   list_matrices.append(melspect)
#   return list_matrices

# Directory path
directory = Path("/content/songs")
file_paths = [file.resolve() for file in directory.iterdir() if file.is_file()]


y, sr = librosa.load(file_paths[0], sr=48000, mono = False)

hop_length_18 = 262144

hop_length_19 = 524288



M = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)

M_db = librosa.power_to_db(M, ref=np.max)
print(np.shape(M_db), np.shape(M))

# Normalize values for display
mel_spec_db_normalized = (M_db - M_db.min()) / (M_db.max() - M_db.min())

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))

img = librosa.display.specshow(mel_spec_db_normalized, y_axis='mel', x_axis='time', ax=ax[0],shading='auto',edgecolors='None')

ax[0].set(title='Mel spectrogram display')
fig.colorbar(img, ax=ax, format="%+2.f dB")


# hop_length = 1024
# D = librosa.amplitude_to_db(np.abs(librosa.stft(y, hop_length=hop_length)),
#                             ref=np.max)
# D_cropped = D[:, :, :3]
# librosa.display.specshow(D_cropped, y_axis='log', sr=sr, hop_length=hop_length,
#                         x_axis='time', ax=ax)
# ax.set(title='Log-frequency power spectrogram')
# ax.label_outer()
# fig.colorbar(img, ax=ax, format="%+2.f dB")

# for i,song_name in enumerate(files):
#   #  song_pieces = prepare_song('song//' + song_name, i)
#   print(i)
#   y, sr = librosa.load('song//' + song_name, sr=48000, mono = False)
#   melspect = librosa.feature.melspectrogram(y=y, sr=sr)
#   melspect_db = librosa.power_to_db(melspect)
#   librosa.display.specshow(melspect_db, x_axis='time', y_axis='mel', ax=ax[i], shading='auto')

#   #  all_tracks += song_pieces

# plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/content/songs'